# YEAR 2025 - Sec 4 Term 2 Class Test

**<font color='red'>
</font>**

# READ THESE INSTRUCTIONS FIRST

Answer **all** questions and **save** your work constantly.

All tasks must be done in the computer laboratory. You are not allowed to bring in or bring out any piece of work, materials, paper or electronic media or in any other form.

The number of marks is given in brackets [ ] at the end of each question or part question.

You are reminded of the need for clear presentation in your answers. Please provide comments where appropriate and ensure the use of descriptive variable and function names.

If any cell is accidentally deleted in the jupyter notebook, you may refer to the html file to recover the content.

You are allowed to add new cells to the notebook, but please make sure to write meaningful comments to explain the purpose.

At the end of the examination, **SAVE** all the changes in the notebook, and save all your source files in the thumb drive and do **NOT** delete your source files.

You are strongly encouraged to manage your time well.

<div style="text-align:right; font-weight:bold">Total Marks: [20]</div>

In [3]:
# Test Run Code Cell:
print("All the best for this paper!")

All the best for this paper!


In [5]:
# Run to link to db and attempt to unlock db when needed.
# if db is still locked, delete the db file and run Task 1.1 to Task 1.3 to reset the db.
import sqlite3
import csv

# !NOTE: if db file does not exist, it will be created
db_file = "reno_contract.db"
conn = sqlite3.connect(db_file)
conn.close()

### Task 1

RenoContract is a company providing a platform for contactors to find workers for home renovation. The company engages you to design a relational database solution.

The following information of each `User` is stored:
- `UserID` – auto increment integer value
- `Name` – name of user
- `Gender` – gender of user, to be stored as a single character, using either `'M'` or `'F'`, default the value to `'M'`
- `Contact` – contact number of user
- `AcctType` – account type of user, can be either `'Contractor'` or `'Worker'`

The following information of each `Job` is stored:
- `JobID` – auto increment integer value to keep track of the ID of a Job
- `JobName` – name of the job
- `Description` – description of the job scope

The following information of each `WorkerJob` is stored:
- `WJID` – auto increment integer value to keep track of the ID of a Worker Job
- `WorkerID` – user ID of worker
- `JobID` – ID of job
- `SkillLevel` – Skill level of this worker for this job, can be `'A'`, `'B'` or `'C'`
- `HourlyRate` – hourly rate of this worker for this job

The following information of each `ContractRecord` is stored:
- `CRID` – auto increment integer value to keep track of the ID of a Contract Record
- `ContractorID` – user ID of contractor
- `WJID` – ID of worker job
- `Location` – address of the contract location
- `Date` – date of the contract
- `StartingTime` – starting time of the contract
- `EndingTime` – ending time of the contract  
*Note: you may assume `Date` and `Time` are all in valid well-formated text string and there's no need to include check constraints.*

#### Task 1.1
Create tables `User`, `Job`, `WorkerJob`, and `ContractRecord` in a SQLite database named `reno_contract.db`.

- The table `User` must use `UserID` as its primary key.
- The table `Job` must use `JobID` as its primary key.
- The table `WorkerJob` must use `WJID` as its primary key, while:
  - `JobID` must refer to `JobID` in `Job` as foreign keys.
  - `WorkerID` must refer to `UserID` in `User` as foreign keys.
- The table `ContractRecord` should use `CRID` as its primary key, while:
  - `ContractorID` must refer to `UserID` in `User`
  - `WJID` must refer to `WJID` in `WorkerJob` as foreign keys.

Please select appropriate **data types**, and include **not null**, **default** and **check** constraints where applicable based on the table specifications.

Record the SQL commands in the cell below.

<div style="text-align:right; font-weight:bold">[6]</div>

In [6]:
# Task 1.1
# consider to use "CREATE TABLE IF NOT EXISTS Tablename",
# to avoid error if table already exists

query1 = """
CREATE TABLE IF NOT EXISTS "User" (
	"UserID"	INTEGER,
	"Name"	TEXT NOT NULL,
	"Contact"	TEXT NOT NULL,
	"Gender"	TEXT NOT NULL DEFAULT 'M' CHECK("Gender" IN ('M', 'F')),
	"AcctType"	TEXT NOT NULL CHECK("AcctType" IN ('Contractor', 'Worker')),
	PRIMARY KEY("UserID" AUTOINCREMENT)
);
"""
query2 = """
CREATE TABLE IF NOT EXISTS "Job" (
	"JobID"	INTEGER,
	"JobName"	TEXT NOT NULL,
	"Description"	TEXT NOT NULL,
	PRIMARY KEY("JobID" AUTOINCREMENT)
);
"""
query3 = """
CREATE TABLE IF NOT EXISTS "WorkerJob" (
	"WJID"	INTEGER,
	"WorkerID"	INTEGER NOT NULL,
	"JobID"	INTEGER NOT NULL,
	"SkillLevel"	TEXT NOT NULL CHECK("SkillLevel" IN ('A', 'B', 'C')),
	"HourlyRate"	REAL NOT NULL,
	FOREIGN KEY("WorkerID") REFERENCES "User"("UserID"),
	FOREIGN KEY("JobID") REFERENCES "Job"("JobID"),
	PRIMARY KEY("WJID" AUTOINCREMENT)
);
"""
query4 = """
CREATE TABLE IF NOT EXISTS "ContractRecord" (
	"CRID"	INTEGER,
	"ContractorID"	INTEGER NOT NULL,
	"WJID"	INTEGER NOT NULL,
	"Location"	TEXT NOT NULL,
    "Date"	TEXT NOT NULL,
	"StartingTime"	TEXT NOT NULL,
	"EndingTime"	TEXT NOT NULL,
	FOREIGN KEY("ContractorID") REFERENCES "User"("UserID"),
	FOREIGN KEY("WJID") REFERENCES "WorkerJob"("WJID"),
	PRIMARY KEY("CRID" AUTOINCREMENT)
);
"""

queries = [query1, query2, query3, query4]

#### Task 1.2

Write Python code to create the database `reno_contract.db` and the tables `User`, `Job`, `WorkerJob`, and `ContractRecord` in the database.

<div style="text-align:right; font-weight:bold">[2]</div>

In [7]:
# Task 1.2

conn = sqlite3.connect(db_file)
for query in queries:
    conn.execute(query)
conn.commit()
conn.close()

#### Task 1.3

The files `users.csv`, `jobs.csv`, `workerjobs.csv` and `records.csv` in the `data_files` folder contain information about users, jobs, worker skills, and contract records. The first row of each file contains the header of the respective columns. Each row in the files is a comma-separated list of values.

Write a Python program to insert all information from the files into the database, `reno_contract.db`.

<div style="text-align:right; font-weight:bold">[7]</div>

In [8]:
table_files = {
    "User": "users.csv",
    "Job": "jobs.csv",
    "WorkerJob": "workerjobs.csv",
    "ContractRecord": "records.csv",
}

# Task 1.3
def read_csv_to_db(table):
    file_name = table_files[table]
    conn = sqlite3.connect(db_file)
    with open(f"data_files/{file_name}", 'r') as f:
        csvreader = csv.reader(f)
        header = next(csvreader)
        placeholders = ",".join(["?" for _ in header])

        for row in csvreader:
            query = f"""
            INSERT INTO {table}
            {tuple(header)}
            VALUES
            ({placeholders})
            """
            try:
                conn.execute(query, tuple(row))
            except sqlite3.Error as e:
                if 'UNIQUE constraint failed' in str(e):
                    # print(f"Row already inserted. Skipping row: {row}")
                    continue
                else:
                    print(f"Error inserting row: {row}")
                    print(e)
            
    conn.commit()
    conn.close()

for table in table_files:
    read_csv_to_db(table)

#### Task 1.4

Implement a function, `search_worker_cr(name)`, to search and display all contract records of a worker. Using the worker's `Name`, such as `'Goh Yi Xi'`, query and display a list of data with the following fields, sorted in ascending order by date:

| Name        | Contact | Date       | StartingTime | EndingTime | JobName   | HourlyRate |
|-------------|---------|------------|--------------|------------|-----------|------------|
| ...         | ...     | ...        | ...          | ...        | ...       | ...        |

Sample Output for `test_task_1_4()`:

```python
|Name        |   Contact|      Date|   StartingTime|EndingTime|   JobName|HourlyRate|
-------------------------------------------------------------------------------------
|Goh Yi Xi   |  99634088|  20210807|           0900|      1800|  Painting|$    32.00|
|Goh Yi Xi   |  99634088|  20210813|           1000|      1800|   Ceiling|$    24.00|
...
```
*Note: the function `display_data()` is provided to display the data in a table format. You are not required to implement the function by yourself.*

<div style="text-align:right; font-weight:bold">[5]</div>

In [9]:
# Task 1.4

def search_worker_cr(name):
    query = """
    SELECT Name, Contact, Date, StartingTime, EndingTime, JobName, HourlyRate
    FROM ContractRecord
    INNER JOIN WorkerJob ON WorkerJob.WJID = ContractRecord.WJID
    INNER JOIN Job ON Job.JobID = WorkerJob.JobID
    INNER JOIN User ON User.UserID = WorkerJob.WorkerID
    WHERE Name = ?
    ORDER BY Date ASC
    """
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    cursor.execute(query, (name,))
    results = cursor.fetchall()
    conn.close()
    return results


In [10]:
# Task 1.4 Test
def display_data(data):
    # | Name        | Contact | Date       | StartingTime | EndingTime | JobName   | HourlyRate |
    header = f"|{'Name':<12}|{'Contact':>10}|{'Date':>10}|{'StartingTime':>15}|{'EndingTime':>10}|{'JobName':>10}|{'HourlyRate':>10}|"
    print(header)
    print("-" * len(header))
    if data is not None:
        for row in data:
            print(f"|{row[0]:<12}|{row[1]:>10}|{row[2]:>10}|{row[3]:>15}|{row[4]:>10}|{row[5]:>10}|${row[6]:>9.2f}|")

def test_task_1_4():
    data = search_worker_cr("Goh Yi Xi")
    display_data(data)

test_task_1_4()

|Name        |   Contact|      Date|   StartingTime|EndingTime|   JobName|HourlyRate|
-------------------------------------------------------------------------------------
|Goh Yi Xi   |  99634088|  20210807|           0900|      1800|  Painting|$    32.00|
|Goh Yi Xi   |  99634088|  20210813|           1000|      1800|   Ceiling|$    24.00|
|Goh Yi Xi   |  99634088|  20210815|           0900|      1600|  Painting|$    32.00|
